# Cross Related Series
* Constructs a graph of anime that are related to manga, or vice-versa

In [ ]:
medium = ""

In [ ]:
using DataFrames
using LinearAlgebra
using SparseArrays
import CSV
import DataFrames: DataFrame
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb");

## Compute similarity matrices

In [ ]:
function get_cross_media(medium)
    if medium == "anime"
        source_media = "manga"
    elseif medium == "manga"
        source_media = "anime"
    else
        @assert false
    end
end

function get_relations(medium, cross_medium)
    file = get_data_path("processed_data/$(medium)_$(cross_medium)_related_series.csv")
    DataFrame(CSV.File(file, stringtype = String))
end

function get_similarity_matrix(medium, cross_medium, relationships)
    df = get_relations(medium, cross_medium)
    df = filter(x -> x.relationship ∈ relationships, df)
    sparse(
        df.target .+ 1,
        df.source .+ 1,
        ones(Float32, length(df.source)),
        num_items(cross_medium),
        num_items(medium),
    )
end

function save_relations(medium, relations, alpha)
    same_media_relations = read_params("$medium/Nondirectional/$alpha", true)["S"]
    cross_media_relations =
        get_similarity_matrix(medium, get_cross_media(medium), relations)
    S = cross_media_relations + cross_media_relations * same_media_relations
    S[S.!=0] .= 1
    write_params(Dict("S" => S), "$medium/Nondirectional/Cross$alpha", true)
end;

In [ ]:
# definitions for the relationships can be found at https://myanimelist.net/info.php?go=relationinfo
const strict_relations = Set(("sequel", "prequel", "parent_story", "side_story"))
const recap_relations = Set(("alternative_version", "summary", "full_story", "adaptation"))
const loose_relations = Set(("alternative_setting", "spin_off"))
const no_relations = Set(("character", "other"))
const all_relations =
    union(strict_relations, recap_relations, loose_relations, no_relations)
@assert all(map(x -> x ∈ all_relations, get_relations(medium, medium).relationship))

In [ ]:
save_relations(
    medium,
    union(strict_relations, recap_relations, loose_relations),
    "RelatedSeries",
);

In [ ]:
save_relations(medium, recap_relations, "RecapSeries");